In [ ]:
import ray
import pandas as pd
import numpy as np
import modules.networker as netw
import modules.network_extractor as net_extractor
import shapely
from shapely import ops
from osmnx import settings
import osmnx as ox
import networkx as nx
import geopandas as geopd
import pyproj

In [ ]:
# Ray is used for pararellizing the download of street network data
ray.shutdown()
ray.init()

In [ ]:
# Extract information from the GHS dataset
urban_centers: geopd.GeoDataFrame

urban_centers = geopd.read_file(
    "/home/geolab/Desktop/Research/data/GHS_URBAN_CENTERS/GHS_URBAN_SIMPLIFIED_fixed.gpkg",
    layer='GHS_URBAN_SIMPLIFIED_fixed'    
)
urban_centers = urban_centers.rename(columns={
    "GC_POP_TOT_2025": "population",
    "GC_UCA_KM2_2025": "area",
    "GC_DEV_USR_2025": "continent",
    "GC_UCN_MAI_2025": "name",
    "GC_CNT_GAD_2025": "country"
})
transform = pyproj.Transformer.from_crs("ESRI:54009", "EPSG:4326", always_xy=True).transform

In [ ]:
# key: the lower case name of the city. This one is used for the folder and identification of the city
# Search: is the name of the city as it appears in GHS
# Country: is the country name as it appears in GHS
# List here all the cities to calculate
cities = {
    "quito": {"search": "Quito", "country": "Ecuador"},
}

stats_dict = {
    "city": [],
    "area": [],
    "population": [],
    "drive_nodes": [],
    "drive_edges": [],
    "bike_nodes": [],
    "bike_edges": [],
    "walk_nodes": [],
    "walk_edges": [],
    "total_time": []
}

In [ ]:
extractor = net_extractor.NetworkExtractor()
# use absolute path to the folder to use
extractor.DATA_BASE_PATH = "/home/geolab/Desktop/Research/data"
networker = netw.Networker()

In [21]:
"""
Extract OSMnx networks for drive | bike | walk 
""" 
dist_threshold=20
slope_threshold=15
assess = False

for key, data in cities.items():
    
    # extract info from GHS
    city_info = urban_centers.loc[
        (urban_centers["name"] == data["search"]) &
        (urban_centers["country"] == data["country"])
    ]

    geometry = city_info["geometry"].values[0]
    geom_reprojected = ops.transform(transform, geometry)

    print(geom_reprojected.bounds)

    
    

(-78.5858108818617, -0.38821782135384897, -78.40536991232686, -0.048517988983367503)


In [ ]:
for key, data in cities.items():
    
    # extract info from GHS
    city_info = urban_centers.loc[
        (urban_centers["name"] == data["search"]) &
        (urban_centers["country"] == data["country"])
    ]

    geometry = city_info["geometry"].values[0]
    geom_reprojected = ops.transform(transform, geometry)
    
    geometry = geom_reprojected
    place = key
    place_DTM = data["search"]

    g_walk = extractor.download_network_sync(
        "walk", 
        geometry, 
        place_DTM, 
        assessment=assess, 
        dist_threshold=dist_threshold, 
        slope_threshold=slope_threshold,
        add_elevation=True,
    )

    # g_promises = []
    # g_promises.append(extractor.download_network.remote(
    #     extractor,
    #     "walk", 
    #     geometry, 
    #     place_DTM, 
    #     assessment=assess, 
    #     dist_threshold=dist_threshold, 
    #     slope_threshold=slope_threshold,
    #     add_elevation=True,
    # ))

    # g_promises.append(extractor.download_network.remote(
    #     extractor,
    #     "bike", 
    #     geometry, 
    #     place_DTM, 
    #     assessment=assess, 
    #     add_elevation=True,
    # ))

    # g_promises.append(extractor.download_network.remote(
    #     extractor,
    #     "drive", 
    #     geometry, 
    #     place_DTM, 
    #     assessment=assess, 
    #     add_elevation=True,
    # ))

    # [g_walk, g_bike, g_drive] = ray.get(g_promises)

    # extractor.save_as_shp(g_walk, f'{place}/shp/walk_{place}')
    # extractor.save_as_shp(g_bike, f'{place}/shp/bike_{place}')
    # extractor.save_as_shp(g_drive, f'{place}/shp/drive_{place}')

    # extractor.save_as_graph(g_walk, f'{place}/graph/walk_{place}')
    # extractor.save_as_graph(g_bike, f'{place}/graph/bike_{place}')
    # extractor.save_as_graph(g_drive, f'{place}/graph/drive_{place}')

    extractor.load_graph()

In [ ]:

extractor.save_as_shp(g_walk, f'{place}/shp/walk_{place}')
extractor.save_as_shp(g_bike, f'{place}/shp/bike_{place}')
extractor.save_as_shp(g_drive, f'{place}/shp/drive_{place}')


In [ ]:
g_promises = []
g_promises.append(extractor.download_network.remote(
    extractor,
    "walk", 
    geometry, 
    place_DTM, 
    assessment=assess, 
    dist_threshold=dist_threshold, 
    slope_threshold=slope_threshold,
    add_elevation=True,
))

g_promises.append(extractor.download_network.remote(
    extractor,
    "bike", 
    geometry, 
    place_DTM, 
    assessment=assess, 
    add_elevation=True,
))

g_promises.append(extractor.download_network.remote(
    extractor,
    "drive", 
    geometry, 
    place_DTM, 
    assessment=assess, 
    add_elevation=True,
))

[g_walk, g_bike, g_drive] = ray.get(g_promises)

extractor.save_as_shp(g_walk, f'{place}/shp/walk_{place}')
extractor.save_as_shp(g_bike, f'{place}/shp/bike_{place}')
extractor.save_as_shp(g_drive, f'{place}/shp/drive_{place}')